### Import

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 13.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import random

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [4]:
CFG = {
    'NBITS':2048,
    'SEED':42,
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG['SEED']) # Seed 고정

### DataLoad

In [6]:
# SMILES 데이터를 분자 지문으로 변환
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=CFG['NBITS'])
        return np.array(fp)
    else:
        return np.zeros((CFG['NBITS'],))

In [7]:
# !unzip '/content/drive/MyDrive/data/신약개발데이터.zip' -d data

Archive:  /content/drive/MyDrive/data/신약개발데이터.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


In [8]:
# 학습 ChEMBL 데이터 로드
chembl_data = pd.read_csv('/content/data/train.csv')  # 예시 파일 이름
chembl_data.head()

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
0,CHEMBL4443947,IC50,'=',0.022,nM,10.66,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,IC50,'=',0.026,nM,10.59,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,IC50,'=',0.078,nM,10.11,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,IC50,'=',0.081,nM,10.09,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,IC50,'=',0.099,nM,10.00,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...


### Data Pre-processing

In [38]:
train = chembl_data[['Smiles', 'pIC50']]
train['Fingerprint'] = train['Smiles'].apply(smiles_to_fingerprint)

train_x = np.stack(train['Fingerprint'].values)
train_y = train['pIC50'].values

# 학습 및 검증 데이터 분리
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.3, random_state=CFG['SEED'])


# 스케일링 적용
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
val_x_scaled = scaler.transform(val_x)


[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:25] DEPRECATION WARNING: please use MorganGenerator
[12:30:26] DEPRECATION WARNING: please use MorganGenerat

In [42]:
train_x_scaled.shape, val_x_scaled.shape

((1366, 2048), (586, 2048))

### Train & Validation

In [39]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# 하이퍼 파라메터 그리드 설정
params = {
    'n_estimators': [int(x) for x in np.linspace(100,300,5)],
    'max_features': ['auto','sqrt'],
    'max_depth': [int(x) for x in np.linspace(10,110,11)]+[None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1,2,4],
    'bootstrap': [True, False]
}

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=CFG['SEED'])

rfr = RandomForestRegressor(random_state=CFG['SEED'])
rcv = RandomizedSearchCV(rfr,param_distributions=params,n_iter=100, cv=kf,verbose=2,random_state=CFG['SEED'],n_jobs=-1)
rcv.fit(train_x_scaled, train_y)

model = rcv.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
270 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

In [30]:
# 랜덤 포레스트 모델 학습
# model = RandomForestRegressor(random_state=CFG['SEED'], **rcv.best_params_)
# model.fit(train_x, train_y)

RandomForestRegressor(bootstrap=False, max_depth=70, max_features='sqrt',
                      min_samples_split=5, n_estimators=200, random_state=42)

In [58]:
# # 랜덤 포레스트 모델 학습
# model = RandomForestRegressor(random_state=CFG['SEED'])
# model.fit(train_x_scaled, train_y)

# from sklearn.ensemble import GradientBoostingRegressor
# model = GradientBoostingRegressor(random_state=CFG['SEED'])


# from xgboost import XGBRegressor
# model = XGBRegressor(n_estimators=100, random_state=CFG['SEED'])

from lightgbm import LGBMRegressor
model = LGBMRegressor(random_state=CFG['SEED'])



model.fit(train_x_scaled, train_y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1698
[LightGBM] [Info] Number of data points in the train set: 1366, number of used features: 566
[LightGBM] [Info] Start training from score 7.527196


LGBMRegressor(random_state=42)

In [59]:
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

# Validation 데이터로부터의 학습 모델 평가
val_y_pred = model.predict(val_x_scaled)
mse = mean_squared_error(pIC50_to_IC50(val_y), pIC50_to_IC50(val_y_pred))
rmse = np.sqrt(mse)

print(f'RMSE: {rmse}')    # RMSE: 2169.5781089857264

RMSE: 2125.480639443984


### Inference

In [60]:
test = pd.read_csv('/content/data/test.csv')
test['Fingerprint'] = test['Smiles'].apply(smiles_to_fingerprint)

test_x = np.stack(test['Fingerprint'].values)
test_x_scaled = scaler.transform(test_x)

test_y_pred = model.predict(test_x_scaled)

[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerator
[12:52:12] DEPRECATION WARNING: please use MorganGenerat

### Submission

In [61]:
submit = pd.read_csv('/content/data/sample_submission.csv')
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit.head()

,ID,IC50_nM
0,TEST_000,84.294194
1,TEST_001,54.267239
2,TEST_002,13.493014
3,TEST_003,11.456870
4,TEST_004,15.523502


In [62]:
submit.to_csv('./baseline_submit3.csv', index=False)